In [7]:
from sklearn import *
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
from sklearn.model_selection import train_test_split
from sqlalchemy import column

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer


import optuna
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm 

In [5]:
stores_train = pd.read_csv('data/stores_train.csv')
plaace_hierarchy = pd.read_csv('data/plaace_hierarchy.csv')
grunnkrets = pd.read_csv('data/grunnkrets_norway_stripped.csv')
grunnkrets_ages = pd.read_csv('data/grunnkrets_age_distribution.csv')
grunnkrets_household_types = pd.read_csv('data/grunnkrets_households_num_persons.csv')
grunnkrets_household_income = pd.read_csv('data/grunnkrets_income_households.csv')

In [ ]:
def convert_to_category(X):
    for col in X.columns:
        #X[col].dtype != 'int64')
        if ( (X[col].dtype != 'int64') and (X[col].dtype != 'float64') and (X[col].dtype != 'bool')):
            X[col] = X[col].astype('category')
            
    return X

def drop_columns(X, columns):
    return X.drop(columns, axis=1)

In [ ]:
stores_train_copy= stores_train.copy()

plaace_hierarchy_copy = plaace_hierarchy.copy()
plaace_hierarchy_copy.drop(columns=['sales_channel_name'], inplace=True)

plaace_hierarchy_copy['lv1'] = plaace_hierarchy_copy['lv1'].astype('object')
plaace_hierarchy_copy['lv2'] = plaace_hierarchy_copy['lv2'].astype('object')

grunnkrets_copy = grunnkrets.copy()
grunnkrets_copy.rename(columns={'year': 'year_1'}, inplace=True)

grunnkrets_ages_copy = grunnkrets_ages.copy()
grunnkrets_ages_copy.rename(columns={'year': 'year_2'}, inplace=True)

grunnkrets_household_types_copy = grunnkrets_household_types.copy()
grunnkrets_household_types_copy.rename(columns={'year': 'year_3'}, inplace=True)

grunnkrets_household_income_copy = grunnkrets_household_income.copy()
grunnkrets_household_income_copy.rename(columns={'year': 'year_4'}, inplace=True)


df = stores_train_copy

df = pd.merge(df, plaace_hierarchy_copy, on='plaace_hierarchy_id', how='left')
df = pd.merge(df, grunnkrets_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_ages_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_types_copy, on='grunnkrets_id', how='left')
df = pd.merge(df, grunnkrets_household_income_copy, on='grunnkrets_id', how='left')

df.drop_duplicates(subset=['store_id'], keep='first', inplace=True)

colums_to_drop = ['revenue', 
                  'store_id',
                  'plaace_hierarchy_id',
                  'grunnkrets_id',
                  ]

X = df.drop(columns=colums_to_drop)
X = convert_to_category(X)
categorical_features = list(X.select_dtypes(include=['category']).columns)

y = df.revenue
y = np.log1p(y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

def objective(trial):
    num_leaves = trial.suggest_int('num_leaves', 45, 65)
    max_depth = trial.suggest_int('max_depth', 80, 100)
    n_estimators = trial.suggest_int('n_estimators', 3500, 3700)
    colsample_bytree = trial.suggest_uniform('colsample_bytree', 0.5, 0.6)
    subsample = trial.suggest_uniform('subsample', 0.8, 0.9)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.04)

    rgr = lgbm.LGBMRegressor(
        verbose=-1,
        n_jobs=8,
        num_leaves=num_leaves,
        max_depth=max_depth,
        random_state=42,
        metric='rmse',
        n_estimators=n_estimators,
        colsample_bytree=colsample_bytree,
        subsample=subsample,
        learning_rate=learning_rate,
        silent=True,
    )

    rgr.fit(X_train, y_train, early_stopping_rounds=100, eval_set=[(X_val, np.log2(y_val))])
    
    print(rgr.get_params())
    lgbm_prediction = rgr.predict(X_val, num_iteration=rgr.best_iteration_)
    lgbm_prediction = np.exp2(lgbm_prediction)

    return np.sqrt(mean_squared_log_error(lgbm_prediction, y_val))

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)
print(study.best_trial)